In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Importing Libraries

In [0]:
import pandas as pd
import math 

Import Dataset

In [5]:
data_temp = pd.read_csv('/content/drive/My Drive/AOBD_Practical_2020/ID3_data.csv', header=None)
data_temp.head()

,0,1,2,3,4
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


Give proper name to the columns

In [6]:
data = data_temp.rename(columns={0:'age',1:'income',2:'student',3:'credit_rating',4:'buys_computer'})
data.head()

,age,income,student,credit_rating,buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes


![alt text](https://drive.google.com/uc?id=1RfX_IHHifmZ6ssdYjSa65u0fZ_u1dcDk)

In [0]:
def calculate_sufficient_statistic(data):


    #calculate number of different values of each attribute
    class_cnt = []
    total_attribute = len(data.columns)
    attr_name = data.columns
    for i in range(total_attribute):
        class_cnt.insert(i,data[attr_name[i]].unique())


    #sufficient statistic column name like age:youth
    suffi_statastic_colname =[]
    for i in range(total_attribute-1):
        cnt = len(class_cnt[i])

        for j in range(cnt):
            suffi_statastic_colname.append(attr_name[i]+':'+class_cnt[i][j])


    #pute all class of attribute in one list like [[age:youth,age:middle_aged,age:senior],...]
    attr_name_class = []
    for i in attr_name:
        attr_name_class.append([col for col in suffi_statastic_colname if i in col])

    attr_name_class=attr_name_class[:-1]
    #print(attr_name_class)

    #convert that into series
    attr_name_class = pd.Series(attr_name_class,index=attr_name[:-1])
    #print(attr_name_class)
        

    #calculate sufficient statistic
    suffi_state =[]
    output_class = class_cnt[-1]
    output_class_name = attr_name[-1]
    for i in range(len(suffi_statastic_colname)):

        temp_list=[]
        str_split = suffi_statastic_colname[i].split(':')
        #print(str_split)
        #data[data[str_split[0]]==str_split[1]]

        for j in output_class:

            temp_list.append(len(data[(data[str_split[0]]==str_split[1]) & (data[output_class_name]==j)]))

        suffi_state.append(temp_list)


    #put all sufficient statistic into dataframe
    temp_list=[]
    for i in output_class:
        temp_list.append(len(data[data[attr_name[-1]]==i]))
    suffi_state.append(temp_list)


    temp=[]
    for i in range(len(output_class)):
        temp.append([x[i] for x in suffi_state])

    #print(temp)
    df_state = pd.DataFrame(temp,columns=suffi_statastic_colname+['output_class'],index = output_class)
    df_state.head()


    return attr_name,attr_name_class,suffi_statastic_colname,df_state



![alt text](https://drive.google.com/uc?id=1pnwuDpprgcr012tiQc2I2tP6XVgfJgh4)

In [0]:
def calculate_giniA(attr,df_suffi_state,attr_class):
    
    giniA = pd.Series()
     
    D = sum(df_suffi_state['output_class'])
    #print('D = ',D)
    
    for i in range(len(attr_class)):
        temp=0
        D1 = sum(df_suffi_state[attr_class[i]])
        D2 = D-D1

        temp+= (D1/D)*( 1 - (df_suffi_state[attr_class[i]][0]/D1)**2 - (df_suffi_state[attr_class[i]][1]/D1)**2)
        temp_sum = [0 for k in range(len(df_suffi_state))]
        for j in attr_class:
            if j == attr_class[i]:
                continue
            for k in range(len(df_suffi_state)):
                temp_sum.insert(k,temp_sum[k]+df_suffi_state[j][k])


        for j in range(len(temp_sum)):
            temp_sum[j] = (temp_sum[j]/D2)**2

        temp+=(D2/D)*(1 - sum(temp_sum))

        giniA[attr_class[i]] = temp
        #print(giniA[attr_class[i]])
        

    #print(info)
    return giniA


In [13]:
[attr_name,attr_name_class,suffi_statastic_colname,df_state] = calculate_sufficient_statistic(data)
calculate_giniA('income',df_state,attr_name_class['income'])

0.4714285714285714
0.38690476190476186
0.3928571428571428


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


income:high      0.471429
income:medium    0.386905
income:low       0.392857
dtype: float64

In [0]:
def best_spliting_attribute(df_state,attr_name_class,attr_name):
    
    #attr_name = df_state.columns

    
    first = True

    for i in attr_name:
        if i == 'buys_computer' or i=='output_class':
            continue
        #print(i)
        #print(attr_name_class[i])
        if first:
            gini = pd.Series(calculate_giniA(i,df_state,attr_name_class[i]))
            first = False
        else:
            gini.append(calculate_giniA(i,df_state,attr_name_class[i]))

    gini_value_list = gini.values.tolist()
    gini_key_list = gini.keys().tolist()
    min_gini_index = gini_value_list.index(min(gini_value_list))

    #print(attr_name[min_info_index])
    best_attr = gini_key_list[min_gini_index]
    #print(best_attr.split(':')[0])
    best_attr_class = [best_attr,best_attr.split(':')[0]]
    best = attr_name_class[best_attr.split(':')[0]]
    for i in best:
        if i == best_attr:
            continue
        best_attr_class[1] = best_attr_class[1] + ':'+i.split(':')[1]
    return best_attr, best_attr_class



In [59]:
best_spliting_attribute(df_state,attr_name_class,attr_name)


age


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


('age:middle_aged', ['age:middle_aged', 'age:youth:senior'])

In [0]:

dic = {}
tree={}

In [0]:

def ID3(data,tree=None):
    
    
    #print('data: ',data)

    [attr_name,attr_name_class,suffi_statastic_colname,df_state] = calculate_sufficient_statistic(data)

    #print('attr name : ',attr_name)
    #print('attr name class : ',attr_name_class)
    #print('suffi_statastic_colname : ',suffi_statastic_colname)
    #print('df_state : ',df_state)
    

    best_attr,best_attr_class = best_spliting_attribute(df_state,attr_name_class,attr_name)
    

    #print('best_attr:',best_attr)

    #best_attr_class = [col for col in suffi_statastic_colname if best_attr in col]

 
 
    if tree is None:                    
        tree={}
        tree[best_attr] = {}

    #print('best_attr_class : ',best_attr_class)
    tree_var = 'yes'
    for attr_class in best_attr_class:
        #print(attr_class)
        temp_data_list=[]
        temp_split = attr_class.split(':')
        for j in range(1,len(temp_split)):

            temp_data_list.append(data[data[temp_split[0]]==temp_split[j]])
            
        temp_data = pd.concat(temp_data_list, ignore_index=True)
        #print('temp_data : ',temp_data)
        temp_data = temp_data.drop([temp_split[0]], axis = 1)

        #print('temp_data : ',temp_data)
        #print('attr_class : ',attr_class)
        if len(temp_data['buys_computer'].unique())>1 and len(temp_data.columns)>2:
            #print('temp_data : ',temp_data)
            #return ID3(temp_data)
            tree[best_attr][tree_var] = ID3(temp_data)
        elif len(temp_data.columns)==2:
            # assign value is whichever maximum from yes and no
            cnt_yes = len(temp_data[temp_data['buys_computer']=='yes'])
            cnt_no = len(temp_data[temp_data['buys_computer']=='no'])
            if cnt_yes >= cnt_no:
                tree[best_attr][tree_var] = 'yes'
            else:
                tree[best_attr][tree_var] = 'no'
        else:
            #print(temp_data['buys_computer'].unique()[0])
            #print(tree)
            tree[best_attr][tree_var] = temp_data['buys_computer'].unique()[0]
        
        tree_var='no'
    
    return tree


In [90]:
ID3(data)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


{'age:middle_aged': {'no': {'income:medium': {'no': {'student:no': {'no': 'yes',
      'yes': 'no'}},
    'yes': {'student:no': {'no': 'yes', 'yes': 'no'}}}},
  'yes': 'yes'}}

In [0]:
#{'age:middle_aged': {'no': {'income:medium': {'no': {'student:no': {'no': 'yes','yes': 'no'}},'yes': {'student:no': {'no': 'yes', 'yes': 'no'}}}},
#                     'yes': 'yes'}}

In [46]:
d = pd.Series(pd.Series([1,2,3]))

p= pd.Series([4,5,6])
d = d.append(p)
print(d)
print(max(d))
print(type(d))
print(d.values.tolist())
print(d.keys().tolist())
a = d.values.tolist()
print(a)
a = a[a!=2]
print(a)

0    1
1    2
2    3
0    4
1    5
2    6
dtype: int64
6
<class 'pandas.core.series.Series'>
[1, 2, 3, 4, 5, 6]
[0, 1, 2, 0, 1, 2]
[1, 2, 3, 4, 5, 6]
2


In [79]:
c = pd.DataFrame()
d = pd.DataFrame({'age': [12,13],'name':['vaishwi','hetvi']})
b = pd.DataFrame({'age': [15],'name':['kusum']})
c = c.append(d)
print(c)
c = c.append(b,ignore_index=True)
print(c)

   age     name
0   12  vaishwi
1   13    hetvi
   age     name
0   12  vaishwi
1   13    hetvi
2   15    kusum


In [0]:
attr_key = 'age:middle_aged'
attr_split = attr_key.split(':')

print(attr_split)
d_temp = d
for key in attr_split:
    print(key)
    d_temp = d_temp[key]
    print(d_temp)
d_temp = 3
print(d)
print(d_temp)

['age', 'middle_aged']
age
{'middle_aged': {}, 'senior': {'hhh': 1}, 'youth': {}}
middle_aged
{}
{'age': {'middle_aged': {}, 'senior': {'hhh': 1}, 'youth': {}}}
3


In [0]:
d['age']['middle_aged']=9

In [0]:
d

{'age': {'middle_aged': 9, 'senior': {'hhh': 1}, 'youth': {}}}